In [ ]:
!pip install ultralytics

In [ ]:
import numpy as np
import cv2
import random
import time
from scipy.spatial.distance import euclidean
from ultralytics import YOLO
from google.colab import files
import matplotlib.colors as mcolors  # Correct import for colors

# Upload file via Colab's file upload functionality
uploaded = files.upload()

# Check if a file is uploaded and get the file name
if uploaded:
    video_path = next(iter(uploaded))  # Get the first uploaded file's name
    print(f"Uploaded file: {video_path}")

    # Load YOLO model
    model = YOLO('/content/drive/MyDrive/best_p6.pt')

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    # Get the video frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # List of 10 different colors
    color_list = [
        "red", "blue", "green", "orange", "purple",
        "pink", "cyan", "brown", "magenta", "yellow"
    ]

    # Notify user about valid number of agents
    print("Please enter the number of agents (between 1 and 10):")
    num_agents = int(input("Enter the number of agents: "))

    # Validate the number of agents
    if num_agents < 1 or num_agents > 10:
        print("Invalid number of agents. Please enter a value between 1 and 10.")
        exit()

    # Initialize agent parameters
    collected_trash = set()
    agent_positions = [(random.randint(0, frame_width), random.randint(0, frame_height)) for _ in range(num_agents)]  # Default resolution
    agent_paths = [[] for _ in range(num_agents)]

    # APF Parameters
    attraction_strength = 1.0
    repulsion_strength = 1.5
    repulsion_distance = 50  # Distance at which agents start to repel each other
    step_size = 5  # Step size for agents' movements

    # Create a VideoWriter object to save the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format
    out = cv2.VideoWriter('waste_with_agents.mp4', fourcc, 20.0, (frame_width, frame_height))  # 20 fps, resolution of video

    # Collect trash for agents with APF and dynamic recalculation
    def dynamic_path_recalculation(trash_coords, num_agents, frame):
            # Create the frame for this step (draw agents and trash)
            frame_with_agents = frame.copy()

            # Draw agents with their unique color
            for agent_idx in range(num_agents):
                agent_color = color_list[agent_idx % len(color_list)]  # Assign color from color_list
                color_rgb = mcolors.to_rgb(agent_color)  # Convert color to RGB format
                color_bgr = tuple([int(c * 255) for c in color_rgb])  # Convert to BGR for OpenCV
                cv2.circle(frame_with_agents, tuple(map(int, agent_positions[agent_idx])), 10, color_bgr, -1)  # Colored circles for agents

            # Draw trash with a fixed color (green)
            for trash in trash_coords:
                cv2.circle(frame_with_agents, tuple(map(int, trash)), 5, (0, 255, 0), -1)  # Green circles for trash

            # Draw the paths with their unique color (per agent)
            for agent_idx in range(num_agents):
                path_coords = [trash_coords[node] for node in agent_paths[agent_idx]]
                for i in range(len(path_coords) - 1):
                    start_point = path_coords[i]
                    end_point = path_coords[i + 1]
                    agent_color = color_list[agent_idx % len(color_list)]  # Assign color from color_list
                    color_rgb = mcolors.to_rgb(agent_color)  # Convert color to RGB format
                    color_bgr = tuple([int(c * 255) for c in color_rgb])  # Convert to BGR for OpenCV
                    cv2.line(frame_with_agents, tuple(map(int, start_point)), tuple(map(int, end_point)), color_bgr, 2)  # Draw path with agent's color

            # Write the frame to the video
            out.write(frame_with_agents)

            if len(collected_trash) == len(trash_coords):
                break

        return agent_paths

    start_time = time.time()  # Start the timer
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB (for YOLO)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform inference to detect trash objects in the current frame
        results = model.predict(source=frame)

        # Initialize a list to store trash coordinates for this frame
        trash_coords = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                class_id = int(box.cls)
                if class_id == 1:
                    x_center, y_center, _, _ = box.xywh[0]
                    trash_coords.append((x_center, y_center))

        # If no trash is found, continue to next frame
        if not trash_coords:
            continue

        # Collect trash for agents in this frame
        agent_paths = dynamic_path_recalculation(trash_coords, num_agents, frame_rgb)

    # Release the video capture object and the VideoWriter object
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print("Video saved as waste_with_agents.mp4")